<a href="https://colab.research.google.com/github/forhow/github-slideshow/blob/main/h_prj01_news_category_classfication_02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 전처리

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
import pickle

In [2]:
pd.set_option('display.unicode.east_asian_width', True)

In [3]:
df = pd.read_csv('/content/datasets/naver_news_titles_210616.csv', index_col=0)
print(df)
print(df.info())

                                                                           title  category
0        김총리  새 거리두기 방안  일상에 큰 변화 될 것                           Politics
1         백신 유급휴가비  지원한다 법개정안 복지위 통과                          Politics
2       쇄신   변화  강조한 송영길  개혁 속도조절  국민공감대 우선 ......         Politics
3        소통수석  열린자세로  의 중요 이웃국 역할할 것  종합 ......              Politics
4                 큐어백  백신 도입     한국  생산 거점                           Politics
...                                                  ...                               ...
28304  2만4000년간 죽지 않은  좀비 가 시베리아에서 나타났다......                       IT
28305   PLAY IT  갤럭시북 경험  키보드에서도 블루투스 키보드  트리오500  써보...        IT
28306   PLAY IT   카카오판 클럽하우스  음  체험기  작지만 큰차이 ......                 IT
28307   PLAY IT  홈쇼핑 고객 정착 가능할까  CJ온스타일  이용해보니......                IT
28308   PLAY IT  크롬 잡겠다는 네이버 웨일  신기능 사용해보니......                     IT

[28309 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28309 entries, 0 to 28308
Dat

In [4]:
# title의 중복확인
col_dup = df['title'].duplicated()
print(col_dup)
sum_dup = df.title.duplicated().sum()
print(sum_dup)

0        False
1        False
2        False
3        False
4        False
         ...  
28304    False
28305    False
28306    False
28307    False
28308    False
Name: title, Length: 28309, dtype: bool
1234


In [5]:
# title의 중복 제거
df = df.drop_duplicates(subset=['title'])
sum_sup = df.title.duplicated().sum()
print(sum_dup)

1234


In [6]:
# 제거된 데이터의 인덱스 재지정
# drop = 기존 인덱스 삭제, False시 기존 인덱스가 새로운 열로 생성됨
df.reset_index(drop=True, inplace=True) 

In [7]:
X = df['title']
Y = df['category']

In [8]:
# Y(target)을- label로 변환
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)

['Culture' 'Economic' 'IT' 'Politics' 'Social' 'World']


In [9]:
# encoding mapping 정보를 저장
with open('/content/datasets/category_encoder.pickle', 'wb') as f:
  pickle.dump(encoder, f)

In [10]:
print(labeled_Y)

[3 3 3 ... 2 2 2]


In [11]:
# label을 onehotencoding 으로 변환
onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [12]:
# Train data 전처리
# - 자연어 처리; 형태소 분석 kokoma, kotlan, Okt

# 형태소 분석 모듈 설치
# !pip install Okt - x
# !pip install konlp - for R
!pip install konlpy

In [13]:
from konlpy.tag import Okt

In [14]:
# 형태소 분리기 마다 성능차이가 있음, 적합한 형태소 분류기를 사용하도록 함
okt = Okt()
print(type(X))
okt_X = okt.morphs(X[0])
print(X[0])
print(okt_X)

<class 'pandas.core.series.Series'>
김총리  새 거리두기 방안  일상에 큰 변화 될 것 
['김', '총리', '새', '거리', '두기', '방안', '일상', '에', '큰', '변화', '될', '것']


In [15]:
# 하나의 문장을 형태소로 분할
for i in range(len(X)):
  X[i] = okt.morphs(X[i])
print(X)

0        [김, 총리, 새, 거리, 두기, 방안, 일상, 에, 큰, 변화, 될, 것]...
1        [백신, 유급, 휴가, 비, 지원, 한, 다, 법, 개정안, 복지, 위, 통과]...
2        [쇄신, 변화, 강조, 한, 송영길, 개혁, 속도, 조절, 국민, 공감, 대, 우선...
3        [소통, 수석, 열린, 자세, 로, 의, 중요, 이웃, 국, 역할, 할, 것, 종합...
4                 [큐어, 백, 백신, 도입, 한국, 생산, 거점]
                               ...                        
27070    [2만, 4000년, 간, 죽지, 않은, 좀비, 가, 시베리아, 에서, 나타났다]...
27071    [PLAY, IT, 갤럭시, 북, 경험, 키, 보드, 에서도, 블루투스, 키, 보드...
27072    [PLAY, IT, 카카오, 판, 클럽, 하우스, 음, 체험, 기, 작, 지만, 큰...
27073    [PLAY, IT, 홈쇼핑, 고객, 정착, 가능할까, CJ, 온스타일, 이용, 해보...
27074    [PLAY, IT, 크롬, 잡겠다는, 네이버, 웨일, 신, 기능, 사용, 해보니]...
Name: title, Length: 27075, dtype: object


In [16]:
# 접속사 조사 감탄사 등 문장분류에 도움이 되지 않는 단어들을 제거
# 불용어(stopwords) 제거 
# stopwords.csv 파일 upload
stopwords = pd.read_csv('/content/datasets/stopwords.csv')

In [17]:
print(stopwords)

     Unnamed: 0 stopword
0             0       아
1             1       휴
2             2   아이구
3             3   아이쿠
4             4   아이고
..          ...      ...
782         783   없다는
783         784   않을까
784         785     있지
785         786   있으며
786         787   그대로

[787 rows x 2 columns]


In [18]:
words = []
for word in okt_X:
  if word not in list(stopwords['stopword']):
    words.append(word)
print(words)
print(len(okt_X))
print(len(words))

['김', '총리', '새', '거리', '두기', '방안', '일상', '큰', '변화', '될']
12
10


In [19]:
# 불용어 제거 후 형태소로 이루어진 문장으로 재조합
for i in range(len(X)) :
  result = []
  for j in range(len(X[i])):
    if len(X[i][j]) > 1:
      if X[i][j] not in list(stopwords['stopword']):
        result.append(X[i][j])
  X[i] = ' '.join(result)

print(X)


0                            총리 거리 두기 방안 일상 변화
1                     백신 유급 휴가 지원 개정안 복지 통과
2           쇄신 변화 강조 송영길 개혁 속도 조절 국민 공감
3                  소통 수석 열린 자세 중요 이웃 역할 종합
4                            큐어 백신 도입 한국 생산 거점
                               ...                        
27070          2만 4000년 죽지 않은 좀비 시베리아 나타났다
27071    PLAY IT 갤럭시 경험 보드 에서도 블루투스 보드 트리오 500 보니...
27072                 PLAY IT 카카오 클럽 하우스 체험 차이
27073    PLAY IT 홈쇼핑 고객 정착 가능할까 CJ 온스타일 이용 해보니...
27074    PLAY IT 크롬 잡겠다는 네이버 웨일 기능 사용 해보니...
Name: title, Length: 27075, dtype: object


In [20]:
# 단어 tokenizing : 각 단어별로 숫자를 배정
token = Tokenizer()
token.fit_on_texts(X) # 형태소에 어떤 숫자를 배정할 것인지 배정
tokened_X = token.texts_to_sequences(X) # 토큰에 저장된 정보를 바탕으로 문장을 변환
print(tokened_X[0])

# tokenizing mapping 정보를 저장해서 차기 자료에도 동일하게 적용할 수 있도록 함

[281, 74, 138, 873, 603, 126]


In [21]:
import pickle  # 데이터 형태 그대로 저장할 수 있도록함

with open('/content/datasets/news_token.pickle', 'wb') as f:
  pickle.dump(token, f)


In [22]:
# 형태소 개수 확인
wordsize = len(token.word_index) +1
print(token.word_index)

print(wordsize)  # index 0를 padding 으로 추가 예정

{'이준석': 1, '백신': 2, '접종': 3, '종합': 4, '윤석열': 5, '코로나': 6, '국민': 7, '만에': 8, '대통령': 9, '확진': 10, '한국': 11, '대선': 12, '광주': 13, '최고': 14, '부동산': 15, '조사': 16, '이재명': 17, '수사': 18, '대표': 19, '정부': 20, '오늘': 21, '삼성': 22, '해야': 23, '사망': 24, '투자': 25, '아파트': 26, '단독': 27, '카카오': 28, '시장': 29, '서울': 30, '붕괴': 31, '중국': 32, '기업': 33, '정치': 34, '사고': 35, '개발': 36, '경찰': 37, '택배': 38, '만원': 39, '바이든': 40, '영상': 41, '비트코인': 42, '사업': 43, '확대': 44, '뉴스': 45, '세계': 46, '미국': 47, 'vs': 48, '논란': 49, '공개': 50, '공급': 51, '이유': 52, '민주당': 53, '의혹': 54, '최대': 55, '신규': 56, '분양': 57, '노조': 58, '인상': 59, '날씨': 60, '금리': 61, '에도': 62, 'sk': 63, '우려': 64, '반도체': 65, '송영길': 66, '추진': 67, '코인': 68, '천안함': 69, 'lg': 70, '현장': 71, '추가': 72, '개월': 73, '거리': 74, '남성': 75, '여성': 76, '시작': 77, '상승': 78, '구속': 79, '주택': 80, '출시': 81, '세대': 82, '내일': 83, '감염': 84, '부산': 85, '국내': 86, '산업': 87, '진자': 88, '사건': 89, '신고': 90, '효과': 91, '청약': 92, '…': 93, '의원': 94, '버스': 95, '기술': 96, '검찰': 97, '참사': 98, '경제': 99, '대출'

In [23]:
# 모델에 제공하는 데이터 길이를 맞춰주기 위한 작업 수행
# 모든 문장 중 가장 긴 문장 기준으로 맞춤
# 문장의 앞자리에 0을 추가함으로써 문장 앞부분의 영향을 미미하도록 유지

# 1. 가장 긴 문자의 길이 확인
max = 0
for i in range(len(tokened_X)):
  if max < len(tokened_X[i]):
    max = len(tokened_X[i])

print(max) # 16

27


In [24]:
# 앞쪽을 0으로 채움
X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   281    74   138
    873   603   126]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     2  4851  1001   105
   1953  2506   189]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0  2507   126  1030    66   636   219
   2825     7  1245]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   923  1803  1804  3229  1421
   1547  1095     4]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0  1805     2   312
     11   156  2661]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    66    19  1139 13780  3725  1139
   1059

In [25]:
# Train / test set split
X_train, X_test, Y_train, Y_test = train_test_split(X_pad, onehot_Y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(24367, 27)
(2708, 27)
(24367, 6)
(2708, 6)


In [26]:
# train / test set을 저장
xy = X_train, X_test, Y_train, Y_test
np.save('/content/datasets/news_data_max_{}_size_{}'.format(max, wordsize), xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
